In [15]:
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO

import pandas as pd
import re as rgx
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

dataset_path = '../../datasets/UScomments.csv'

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shakabrah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
df = pd.read_csv(dataset_path, error_bad_lines=False)

/home/shakabrah/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'
b'Skipping line 388430: expected 4 fields, saw 5\n'
/home/shakabrah/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
df.shape

(691400, 4)

In [18]:
df.dtypes

video_id        object
comment_text    object
likes           object
replies         object
dtype: object

In [19]:
df.likes = df.likes.astype(str)

In [22]:
df.replies.unique()

array(['0', '1', '500', '2', '261', '29', '180', '3', '4', '5', '77', '6',
       '14', '8', '11', '15', '7', '10', '156', '9', '13', '20', '16',
       '85', '520', '83', '22', '12', '24', '34', '240', '504', '32',
       '31', '196', '104', '50', '44', '138', '95', '21', '351', '105',
       '19', '166', '471', '266', '46', '136', '66', '286', '208', '259',
       '18', '112', '168', '28', '36', '88', '205', '78', '55', '164',
       '101', '39', '17', '86', '177', '40', '322', '291', '335', '495',
       '476', 'replies', '42', '159', '27', '478', '92', '503', '26',
       '25', '161', '242', '38', '371', '60', '354', '401', '33', '501',
       '49', '47', '224', '71', '54', '23', '65', '41', '80', '91', '311',
       '45', '230', '140', '529', '485', '246', '94', '412', '79', '486',
       '292', '391', '323', '376', '132', '488', '453', '264', '160',
       '169', '528', '282', '149', '466', '74', '499', '225', '370',
       '297', '329', '73', '175', '477', '181', '277', '301', '

In [20]:
df.dtypes

video_id        object
comment_text    object
likes           object
replies         object
dtype: object

In [7]:
df 

,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0
...,...,...,...,...
691395,EoejGgUNmVU,Лучшая,1,0
691396,EoejGgUNmVU,qu'est ce que j'aimerais que tu viennes à Roan...,0,0
691397,EoejGgUNmVU,Ven a mexico! 😍 te amo LP,0,0
691398,EoejGgUNmVU,Islığı yeter...,0,0


# Find Hashtags

In [8]:
def hashtags(text):
    """
    text : String
    return : String List -> ona verilen text'in içindeki hashtagleri liste halinde döndürür.
    """
    return str(rgx.findall(r'#(\w+)', text))

In [9]:
sampletweet = '#covid2020 #vaccines are being made #2020'
hashtags(sampletweet)

"['covid2020', 'vaccines', '2020']"

In [10]:
df.comment_text[95]

'#1 trending whoaaaa'

# Emojies

In [11]:
def emoji(text):
    for emot in UNICODE_EMOJI:
        if text == None:
            text= text
        else:
            text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",", "").replace(":", "").split()))
    return text

In [12]:
temp = "❤ you"
emoji(temp)

'red_heart you'

# Remove @usernames

In [13]:
def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = rgx.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
    # remove tweeted at
    return tweet

In [14]:
temp = '@username1 hi there @username, wazap?'
remove_users(temp)

' hi there , wazap?'

# Remove links

In [15]:
def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = rgx.sub(r'http\S+', '', tweet) # remove http links
    tweet = rgx.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

In [16]:
def clean_html(text):
  html = rgx.compile('<.*?>')#regex
  return html.sub(r'',text)

# Remove non-ascii chars for English language only

In [17]:
def non_ascii(s):
  return "".join(i for i in s if ord(i)<128)

def lower(text):
  return text.lower()

# Stopwords

In [18]:
def removeStopWords(str):
#select english stopwords
  cachedStopWords = set(stopwords.words("english"))
#add custom words
  cachedStopWords.update(('and','I','A','http','And','So','arnt','This','When','It','many','Many','so','cant','Yes','yes','No','no','These','these','mailto','regards','ayanna','like','email'))
#remove stop words
  new_str = ' '.join([word for word in str.split() if word not in cachedStopWords]) 
  return new_str

# Email adresses

In [19]:
def email_address(text):
  email = rgx.compile(r'[\w\.-]+@[\w\.-]+')
  return email.sub(r'',text)

def punct(text):
  token=RegexpTokenizer(r'\w+')#regex
  text = token.tokenize(text)
  text= " ".join(text)
  return text 

In [20]:
def remove_digits(text):
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    return rgx.sub(pattern, '', text) 

def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return rgx.sub(pat, '', text)

def remove_(tweet):
    tweet = rgx.sub( '([_]+)', '', tweet )
    return tweet

In [18]:

df['new_comment_text'] = df.comment_text.apply(func = emoji)
df['new_comment_text'] = df.new_comment_text.apply(func = remove_users)
df['new_comment_text'] = df.new_comment_text.apply(func = clean_html)
df['new_comment_text'] = df.new_comment_text.apply(func = non_ascii)
df['hashtag'] = df.new_comment_text.apply(func = hashtags)
df['new_comment_text'] = df.new_comment_text.apply(func = lower)
df['new_comment_text'] = df.new_comment_text.apply(func = email_address)
df['new_comment_text'] = df.new_comment_text.apply(func = removeStopWords)
df['new_comment_text'] = df.new_comment_text.apply(func = clean_html)
df['new_comment_text'] = df.new_comment_text.apply(func = punct)
df['new_comment_text'] = df.new_comment_text.apply(func = remove_)



AttributeError: 'float' object has no attribute 'replace'